# Model


In [32]:
import shutil
import urllib.request
import os
import cv2
import numpy as np
import yaml
import gdown

from pathlib import Path
from typing import *

In [33]:
models_folder = Path("models")

models_folder.mkdir(parents=True, exist_ok=True)
(models_folder / "__init__.py").touch()

Each tracker must inherit from the `Tracker` class.


In [34]:
%%writefile model.py
import os
from typing import *
import cv2

class Tracker:
    def __init__(self, model_name: str):
        self.model_name: str = model_name

    def initialize(self, image_file, box):
        """Initialize the tracker with the first frame and the bounding box."""
        pass

    def track(self, image_file):
        """Track the object in the next frame and return the new bounding box."""
        pass

Overwriting model.py


In [35]:
def clone_repo(model: str, url: str):
    """Clone a repo from GitHub and put it in the models folder."""
    path_zip: Path = models_folder / (model + ".zip")
    dest_folder: Path = models_folder / model

    if dest_folder.exists():
        return

    urllib.request.urlretrieve(url, path_zip)
    shutil.unpack_archive(path_zip, models_folder)

    try:
        (models_folder / (model + "-master")).rename(dest_folder)
    except:
        (models_folder / (model + "-main")).rename(dest_folder)

    path_zip.unlink()

    (dest_folder / "__init__.py").touch()

## [Staple](https://arxiv.org/pdf/1512.01355v2.pdf)

In [316]:
model = "pyCFTrackers"
url = "https://github.com/noTban/pyCFTrackers/archive/refs/heads/master.zip"

clone_repo(model, url)

In [317]:
%cd models/pyCFTrackers
# install region
%cd libs/eco/features/
!python setup.py build_ext --inplace
%cd ../../..

%cd libs/pysot/utils/
!python setup.py build_ext --inplace
%cd ../../..

%cd ../..

/mnt/d/Documents/DNN_project/projet/models/pyCFTrackers
/mnt/d/Documents/DNN_project/projet/models/pyCFTrackers/libs/eco/features
/mnt/d/Documents/DNN_project/projet/models/pyCFTrackers/libs/eco/features/setup.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.core import setup, Extension
/mnt/d/Documents/DNN_project/projet/models/pyCFTrackers/libs/eco/features/setup.py:3: DeprecationWarning: 

  `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
  of the deprecation of `distutils` itself. It will be removed for
  Python >= 3.12. For older Python versions it will remain present.
  It is recommended to use `setuptools < 60.0` for those Python versions.
  For more details, see:
    https://numpy.org/devdocs/reference/distutils_status_migration.html 


  from numpy.distutils import misc_util
running build_ext
/mnt/d/Documents/DNN_project/projet/models/

In [318]:
%%file -a model.py

def get_Staple_tracker():
    os.sys.path.append('models/pyCFTrackers')
    from cftracker.staple import Staple
    from cftracker.config import staple_config
    class StapleTracker(Tracker):
        def __init__(self):
            super().__init__("Staple")
            self.tracker = Staple(config=staple_config.StapleConfig())

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            self.tracker.init(image, box)

        def track(self, image_file):
            image = cv2.imread(image_file)
            bbox = self.tracker.update(image)
            return bbox
    
    return StapleTracker()

Appending to model.py


## [SiamSE](https://github.com/isosnovik/SiamSE)


In [319]:
model = "SiamSE"
repo_url = "https://github.com/ISosnovik/SiamSE/archive/refs/heads/master.zip"
clone_repo(model, repo_url)

weight_path = str(models_folder / "SiamSE" / "checkpoint_vot.pth")

if not os.path.exists(weight_path):
    weight_url = "https://drive.google.com/uc?id=1WQ-9_QE9Xk9wj52vVcEDIXY2NBTupAnZ"
    gdown.download(weight_url, output=weight_path)

In [320]:
%%writefile -a model.py

import os
import cv2
import yaml
import numpy as np
import models.SiamSE.lib.models.models as models

from pathlib import Path
from models.SiamSE.lib.tracker import SESiamFCTracker
from models.SiamSE.lib.utils import load_pretrain, cxy_wh_2_rect, convert_color_RGB
from typing import *

def get_SESIAMFC_tracker():
    
    def get_axis_aligned_bbox(bbox):
        """Convert bbox to [xc, yc, w, h] format"""
        x, y, w, h = bbox
        return [x + w / 2, y + h / 2, w, h]
    
    class SEsiamfc(Tracker):
        def __init__(self):
            super().__init__("SEsiamfc")

            path_SiamSE = Path('models') / 'SiamSE'
            model_pretrained_path: Path = path_SiamSE / 'checkpoint_vot.pth'
            config_path: Path = path_SiamSE / 'configs' / 'test.yaml'
            
            with open(config_path, 'r') as f:
                tracker_config: Dict = yaml.load(f.read(), Loader=yaml.FullLoader)

            # Prepare model
            net: Any = models.__dict__[tracker_config['MODEL']](padding_mode='constant')
            net = load_pretrain(net, model_pretrained_path)
            net = net.eval().cuda()

            # Prepare tracker
            tracker_config: Dict = tracker_config['TRACKER']['VOT2017']
            self.tracker = SESiamFCTracker(net, **tracker_config)

        def initialize(self, image_file, box):
            # Prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # Prepare box
            cx, cy, w, h = get_axis_aligned_bbox(box)
            target_pos = np.array([cx, cy])
            target_sz = np.array([w, h])

            self.tracker.init(image, target_pos, target_sz)

        def track(self, image_file):
            # Prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # Track
            target_pos, target_sz = self.tracker.track(image)

            # Prepare bbox
            return cxy_wh_2_rect(target_pos, target_sz)
    
    return SEsiamfc()

Appending to model.py


## [PyECO]((https://github.com/StrangerZhang/pyECO)) - Hand-crafted features

In [321]:
model = "pyECO"
url = "https://github.com/StrangerZhang/pyECO/archive/refs/heads/master.zip"

clone_repo(model, url)

In [322]:
if not (models_folder / model / "eco" / "features" / "_gradient.cpython-310-x86_64-linux-gnu.so").exists():
    %cd models/pyECO/eco/features/
    !python setup.py build_ext --inplace
    %cd -

In [323]:
%%file -a model.py

def get_PyECO_tracker(model_name="pyECO"):
    os.sys.path.append('models/' + model_name)
    from eco import ECOTracker
    
    class PyECO(Tracker):
        def __init__(self):
            super().__init__(model_name)
            self.tracker = ECOTracker(True)

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            self.tracker.init(image, box)

        def track(self, image_file):
            image = cv2.imread(image_file)
            extremities = self.tracker.update(image)
            bbox = [extremities[0], extremities[1], extremities[2] - extremities[0], extremities[3] - extremities[1]]
            return bbox
    
    return PyECO()

Appending to model.py


## [PyECO]((https://github.com/StrangerZhang/pyECO)) - Deep features

In [324]:
model = "pyECO_deep"
url = "https://github.com/noTban/pyECO_deep/archive/refs/heads/master.zip"

clone_repo(model, url)

In [325]:
if not (models_folder / model / "eco" / "features" / "_gradient.cpython-310-x86_64-linux-gnu.so").exists():
    %cd models/pyECO_deep/eco/features/
    !python setup.py build_ext --inplace
    %cd -

## [MixFormer](https://github.com/MCG-NJU/MixFormer)

In [326]:
model = "MixFormer"
url = "https://github.com/Sefray/MixFormer/archive/refs/heads/master.zip"

clone_repo(model, url)

weight_file = 'mixformer_online_22k.pth.tar'
weight_path = str(models_folder / "MixFormer" / weight_file)

if not os.path.exists(weight_path):
    weight_url = "https://drive.google.com/uc?id=1-UonqFXM-jKNnkJmN8lSEczeaKiNVxeh"
    gdown.download(weight_url, output=weight_path)

    %cd models/MixFormer/
    !python tracking/create_default_local_file.py --workspace_dir . --data_dir ./data --save_dir .
    %cd -

In [327]:
%%file -a model.py

def get_MixFormer_tracker():
    os.sys.path.append('models/MixFormer')
    from lib.test.tracker.mixformer import MixFormer as MF
    import importlib
    class MixFormer(Tracker):
        def __init__(self):
            super().__init__("MixFormer")
            
            tracker_params = {}
            tracker_params['model'] = 'mixformer_online_22k.pth.tar'
            tracker_params['max_score_decay'] = 1.0
            tracker_params['vis_attn'] = 0

            param_module = importlib.import_module('lib.test.parameter.mixformer')
            search_area_scale = None
            if tracker_params is not None and 'search_area_scale' in tracker_params:
                search_area_scale = tracker_params['search_area_scale']
            model = ''
            if tracker_params is not None and 'model' in tracker_params:
                model = tracker_params['model']
            params = param_module.parameters('baseline', model, search_area_scale)
            if tracker_params is not None:
                for param_k, v in tracker_params.items():
                    setattr(params, param_k, v)
            
            self.tracker = MF(params)

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            info = {'init_bbox': box}
            self.tracker.initialize(image, info)

        def track(self, image_file):
            image = cv2.imread(image_file)
            info = self.tracker.track(image)
            return info['target_bbox']

    return MixFormer()

Appending to model.py


## [MDNet](https://github.com/hyeonseobnam/py-MDNet)

In [328]:
model = "py-MDNet"
url = "https://github.com/noTban/py-MDNet/archive/refs/heads/master.zip"

If you don't have a GPU, change *use_gpu* to *false* in models/py-MDNet/tracking/options.yaml

In [329]:
clone_repo(model, url)

In [330]:
%%file -a model.py

def get_pyMDNet_tracker():
    os.sys.path.append('models/py-MDNet')
    from modules.model import MDNet, BCELoss, set_optimizer
    from modules.sample_generator import SampleGenerator
    from modules.utils import overlap_ratio
    from tracking.bbreg import BBRegressor
    from tracking.data_prov import RegionExtractor
    from tracking.gen_config import gen_config
    import cv2
    import yaml
    import torch
    from PIL import Image
    import numpy as np
    
    
    class pyMDNet(Tracker):
        def __init__(self):
            super().__init__("pyMDNet")
            self.opts = yaml.safe_load(open('models/py-MDNet/tracking/options.yaml','r'))
            self.tracker = MDNet('models/py-MDNet/models/mdnet_imagenet_vid.pth')
            if self.opts['use_gpu']:
                self.tracker = self.tracker.cuda()
            self.target_bbox = None
            self.sample_generator = None
            self.pos_generator = None
            self.neg_generator = None
            self.pos_feats_all = None
            self.neg_feats_all = None
            self.bbreg = None
            self.criterion = BCELoss()
            self.i = 0
            
        def forward_samples(self, image, samples, out_layer='conv3'):
            self.tracker.eval()
            extractor = RegionExtractor(image, samples, self.opts)
            for i, regions in enumerate(extractor):
                if self.opts['use_gpu']:
                    regions = regions.cuda()
                with torch.no_grad():
                    feat = self.tracker(regions, out_layer=out_layer)
                if i==0:
                    feats = feat.detach().clone()
                else:
                    feats = torch.cat((feats, feat.detach().clone()), 0)
            return feats
   
        def train(self, optimizer, pos_feats, neg_feats, maxiter, in_layer='fc4'):
            self.tracker.train()

            batch_pos = self.opts['batch_pos']
            batch_neg = self.opts['batch_neg']
            batch_test = self.opts['batch_test']
            batch_neg_cand = max(self.opts['batch_neg_cand'], batch_neg)

            pos_idx = np.random.permutation(pos_feats.size(0))
            neg_idx = np.random.permutation(neg_feats.size(0))
            while(len(pos_idx) < batch_pos * maxiter):
                pos_idx = np.concatenate([pos_idx, np.random.permutation(pos_feats.size(0))])
            while(len(neg_idx) < batch_neg_cand * maxiter):
                neg_idx = np.concatenate([neg_idx, np.random.permutation(neg_feats.size(0))])
            pos_pointer = 0
            neg_pointer = 0

            for _ in range(maxiter):

                # select pos idx
                pos_next = pos_pointer + batch_pos
                pos_cur_idx = pos_idx[pos_pointer:pos_next]
                pos_cur_idx = pos_feats.new(pos_cur_idx).long()
                pos_pointer = pos_next

                # select neg idx
                neg_next = neg_pointer + batch_neg_cand
                neg_cur_idx = neg_idx[neg_pointer:neg_next]
                neg_cur_idx = neg_feats.new(neg_cur_idx).long()
                neg_pointer = neg_next

                # create batch
                batch_pos_feats = pos_feats[pos_cur_idx]
                batch_neg_feats = neg_feats[neg_cur_idx]

                # hard negative mining
                if batch_neg_cand > batch_neg:
                    self.tracker.eval()
                    for start in range(0, batch_neg_cand, batch_test):
                        end = min(start + batch_test, batch_neg_cand)
                        with torch.no_grad():
                            score = self.tracker(batch_neg_feats[start:end], in_layer=in_layer)
                        if start==0:
                            neg_cand_score = score.detach()[:, 1].clone()
                        else:
                            neg_cand_score = torch.cat((neg_cand_score, score.detach()[:, 1].clone()), 0)

                    _, top_idx = neg_cand_score.topk(batch_neg)
                    batch_neg_feats = batch_neg_feats[top_idx]
                    self.tracker.train()

                # forward
                pos_score = self.tracker(batch_pos_feats, in_layer=in_layer)
                neg_score = self.tracker(batch_neg_feats, in_layer=in_layer)

                # optimize
                loss = self.criterion(pos_score, neg_score)
                self.tracker.zero_grad()
                loss.backward()
                if 'grad_clip' in self.opts:
                    torch.nn.utils.clip_grad_norm_(self.tracker.parameters(), self.opts['grad_clip'])
                optimizer.step()

        def initialize(self, image_file, box):
            self.i += 1
            image = Image.open(image_file).convert('RGB')

            self.target_bbox = np.array(box)

            # Init self.criterion and optimizer 
            self.tracker.set_learnable_params(self.opts['ft_layers'])
            init_optimizer = set_optimizer(self.tracker, self.opts['lr_init'], self.opts['lr_mult'])
            self.update_optimizer = set_optimizer(self.tracker, self.opts['lr_update'], self.opts['lr_mult'])

            # Draw pos/neg samples
            pos_examples = SampleGenerator('gaussian', image.size, self.opts['trans_pos'], self.opts['scale_pos'])(
                                self.target_bbox, self.opts['n_pos_init'], self.opts['overlap_pos_init'])

            neg_examples = np.concatenate([
                            SampleGenerator('uniform', image.size, self.opts['trans_neg_init'], self.opts['scale_neg_init'])(
                                self.target_bbox, int(self.opts['n_neg_init'] * 0.5), self.opts['overlap_neg_init']),
                            SampleGenerator('whole', image.size)(
                                self.target_bbox, int(self.opts['n_neg_init'] * 0.5), self.opts['overlap_neg_init'])])
            neg_examples = np.random.permutation(neg_examples)

            # Extract pos/neg features
            pos_feats = self.forward_samples(image, pos_examples)
            neg_feats = self.forward_samples(image, neg_examples)

            # Initial training
            self.train(init_optimizer, pos_feats, neg_feats, self.opts['maxiter_init'])
            del init_optimizer, neg_feats
            torch.cuda.empty_cache()

            # Train bbox regressor
            bbreg_examples = SampleGenerator('uniform', image.size, self.opts['trans_bbreg'], self.opts['scale_bbreg'], self.opts['aspect_bbreg'])(
                                self.target_bbox, self.opts['n_bbreg'], self.opts['overlap_bbreg'])
            bbreg_feats = self.forward_samples(image, bbreg_examples)
            self.bbreg = BBRegressor(image.size)
            self.bbreg.train(bbreg_feats, bbreg_examples, self.target_bbox)
            del bbreg_feats
            torch.cuda.empty_cache()

            # Init sample generators for update
            self.sample_generator = SampleGenerator('gaussian', image.size, self.opts['trans'], self.opts['scale'])
            self.pos_generator = SampleGenerator('gaussian', image.size, self.opts['trans_pos'], self.opts['scale_pos'])
            self.neg_generator = SampleGenerator('uniform', image.size, self.opts['trans_neg'], self.opts['scale_neg'])

            # Init pos/neg features for update
            neg_examples = self.neg_generator(self.target_bbox, self.opts['n_neg_update'], self.opts['overlap_neg_init'])
            neg_feats = self.forward_samples(image, neg_examples)
            self.pos_feats_all = [pos_feats]
            self.neg_feats_all = [neg_feats]

        def track(self, image_file):
            self.i +=1
            image = Image.open(image_file).convert('RGB')

            # Estimate target bbox
            samples = self.sample_generator(self.target_bbox, self.opts['n_samples'])
            sample_scores = self.forward_samples(image, samples, out_layer='fc6')

            top_scores, top_idx = sample_scores[:, 1].topk(5)
            top_idx = top_idx.cpu()
            target_score = top_scores.mean()
            self.target_bbox = samples[top_idx]
            if top_idx.shape[0] > 1:
                self.target_bbox = self.target_bbox.mean(axis=0)
            success = target_score > 0
                
            # Expand search area at failure
            if success:
                self.sample_generator.set_trans(self.opts['trans'])
            else:
                self.sample_generator.expand_trans(self.opts['trans_limit'])

            # Bbox regression
            if success:
                bbreg_samples = samples[top_idx]
                if top_idx.shape[0] == 1:
                    bbreg_samples = bbreg_samples[None,:]
                bbreg_feats = self.forward_samples(image, bbreg_samples)
                bbreg_samples = self.bbreg.predict(bbreg_feats, bbreg_samples)
                bbreg_bbox = bbreg_samples.mean(axis=0)
            else:
                bbreg_bbox = self.target_bbox

            # Data collect
            if success:
                pos_examples = self.pos_generator(self.target_bbox, self.opts['n_pos_update'], self.opts['overlap_pos_update'])
                pos_feats = self.forward_samples(image, pos_examples)
                self.pos_feats_all.append(pos_feats)
                if len(self.pos_feats_all) > self.opts['n_frames_long']:
                    del self.pos_feats_all[0]

                neg_examples = self.neg_generator(self.target_bbox, self.opts['n_neg_update'], self.opts['overlap_neg_update'])
                neg_feats = self.forward_samples(image, neg_examples)
                self.neg_feats_all.append(neg_feats)
                if len(self.neg_feats_all) > self.opts['n_frames_short']:
                    del self.neg_feats_all[0]

            # Short term update
            if not success:
                nframes = min(self.opts['n_frames_short'], len(self.pos_feats_all))
                pos_data = torch.cat(self.pos_feats_all[-nframes:], 0)
                neg_data = torch.cat(self.neg_feats_all, 0)
                self.train( self.update_optimizer, pos_data, neg_data, self.opts['maxiter_update'])

            # Long term update
            elif self.i % self.opts['long_interval'] == 0:
                pos_data = torch.cat(self.pos_feats_all, 0)
                neg_data = torch.cat(self.neg_feats_all, 0)
                self.train( self.update_optimizer, pos_data, neg_data, self.opts['maxiter_update'])

            torch.cuda.empty_cache()

            return bbreg_bbox
    
    return pyMDNet()

Appending to model.py


## [MIL](https://ieeexplore.ieee.org/document/5206737)

In [331]:
model = "MIL"
url = "https://github.com/noTban/py-MDNet/archive/refs/heads/master.zip"
clone_repo

In [332]:
%%file -a model.py

def get_MIL_tracker():
    import cv2
    class MIL(Tracker):
        def __init__(self):
            super().__init__("MIL")
            self.tracker = cv2.TrackerMIL_create()

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            self.tracker.init(image, box)

        def track(self, image_file):
            image = cv2.imread(image_file)
            _, bbox = self.tracker.update(image)
            return bbox
  
    return MIL()

Appending to model.py


## GOTURN

In [29]:
model = "goturn-files"
url = "https://github.com/spmallick/goturn-files/archive/refs/heads/master.zip"
clone_repo(model, url)

In [31]:
%cd models/goturn-files
!cat goturn.caffemodel.zip* > goturn.caffemodel.zip
%cd ../../
!unzip models/goturn-files/goturn.caffemodel.zip
!cp models/goturn-files/goturn.prototxt .

/home/marius/epita/ING3/DNN/DNN/models/goturn-files
/home/marius/epita/ING3/DNN/DNN
Archive:  models/goturn-files/goturn.caffemodel.zip
  inflating: goturn.caffemodel       


In [36]:
%%file -a model.py

def get_GOTURN_tracker():
    import cv2
    class GOTURN(Tracker):
        def __init__(self):
            super().__init__("GOTURN")
            self.tracker = cv2.TrackerGOTURN_create()

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            self.tracker.init(image, box)

        def track(self, image_file):
            image = cv2.imread(image_file)
            _, bbox = self.tracker.update(image)
            return bbox
  
    return GOTURN()

Appending to model.py


## [AAA](https://github.com/songheony/A3T)

In [333]:
model = "A3T"
url = "https://github.com/Leiyks/A3T/archive/refs/heads/master.zip"

clone_repo(model, url)

This tracker is a bagging tracker. It uses the tracks of other trackers to make its own predictions. In our testing, we will try to see if the trackers using all the trackers we are testing is better then them individually.


In [334]:
if not (models_folder / "A3T" / "pysot-toolkit").exists():
    %cd models/A3T

    # clone frameworks
    !git clone https://github.com/songheony/pytracking.git
    !git clone https://github.com/StrangerZhang/pysot-toolkit

    # install region
    %cd pysot-toolkit/pysot/utils/
    !python setup.py build_ext --inplace
    %cd ../../../../../	


Some modification on the path used has to be performed.


In [335]:
%%writefile -a model.py

def get_AAA_tracker():
    import numpy as np
    class AAA(Tracker):
        def __init__(self):
            super().__init__("AAA")
            import models.A3T.algorithms.aaa as aaa
            self.experts = [
                get_SESIAMFC_tracker(),
                get_PyECO_tracker(),
                get_PyECO_tracker(model_name='PyECO_deep'),
                get_MixFormer_tracker(),
                get_MIL_tracker(),
                get_Staple_tracker(),
                get_GOTURN_tracker(),
            ]
            self.n_experts = len(self.experts) # number of experts
            theta, gamma = 0.92, 11  # you can tune hyperparameters by running run_tuning.sh
            self.tracker = aaa.AAA(self.n_experts, mode="LOG_DIR", threshold=theta)


        def initialize(self, image_path, bbox):
            for i in range(self.n_experts):
                self.experts[i].initialize(image_path, bbox)
            self.tracker.initialize(image_path, bbox)

        def track(self, image_path):
            experts_result = np.zeros((self.n_experts, 4))
            for i in range(self.n_experts):
                experts_result[i, :] = self.experts[i].track(image_path)
            state, offline, weight = self.tracker.track(image_path, experts_result)
            return state
  
    return AAA()

Appending to model.py


# All


In [336]:
print(os.getcwd())

/mnt/d/Documents/DNN_project/projet


In [37]:
%%writefile -a model.py

def load_models() -> Dict[str, Tracker]:
    """Load all models
    Returns:
        Dict[str, Tracker]: Dict of models
    """
    ret: Dict[str, Tracker] = {}
    
    try:
        ret['Staple'] = get_Staple_tracker()
    except:
        ret['Staple'] = get_Staple_tracker()
    ret['SEsiamFC'] = get_SESIAMFC_tracker()
    ret['AAA'] = get_AAA_tracker()
    ret['PyECO'] = get_PyECO_tracker()
    ret['PyECO_deep'] = get_PyECO_tracker(model_name='PyECO_deep')
    ret['MixFormer'] = get_MixFormer_tracker()
    # ret['pyMDNet'] = get_pyMDNet_tracker()
    ret['MIL'] = get_MIL_tracker()
    ret['GOTURN'] = get_GOTURN_tracker()
    
    return ret

Appending to model.py


#
